In [1]:
import numpy as np
from scipy import stats

In [24]:
class BookSCM:
    def __init__(self, random_seed=None):
        self.random_seed = random_seed
        self.u_0 = stats.uniform()
        self.u_1 = stats.norm()

    def sample(self, sample_size=100):
        if self.random_seed:
            np.random.seed(self.random_seed)
        u_0 = self.u_0.rvs(sample_size)
        u_1 = self.u_1.rvs(sample_size)
        a = u_0 > .61
        b = (a + .5 * u_1) > .2
        return a, b


In [25]:
scm = BookSCM(random_seed=45)
buy_book_a, buy_book_b = scm.sample(100)

## Association

In [26]:
buy_book_a[buy_book_b].sum() / buy_book_a[buy_book_b].shape[0]

0.6379310344827587

# Intervention

In [27]:
SAMPLE_SIZE = 100
np.random.seed(45)

u_0 = np.random.randn(SAMPLE_SIZE)
u_1 = np.random.randn(SAMPLE_SIZE)
a = u_0
b = 5 * a + u_1
r, p = stats.pearsonr(a, b)

print(f"mean of B before any intervention: {b.mean():.3f}")
print(f"variance of B before any intervention: {b.var():.3f}")
print(f"corr between A and B: r={r:.3f}; p={p:.3f}")

mean of B before any intervention: -0.620
variance of B before any intervention: 22.667
corr between A and B: r=0.978; p=0.000


In [28]:
a = np.array([1.5] * SAMPLE_SIZE)
b = 5 * a + u_1

print(b.mean())
print(b.var())

7.575404468180015
1.0025411156718491


In [29]:
a = u_0
b = np.random.randn(SAMPLE_SIZE)
r, p = stats.pearsonr(a, b)

print(b.mean(), b.var(), r, p)

0.18551970084448796 0.9949738521691471 -0.022866039104679478 0.82134525059407


# Conterfactual

$P(Y_{X=0} = 1| X = 1, Y_{X=1} = 1)$
- probabilidade do outcome ser positivo caso o tratamento tenha sido negativo ($P(Y_{X=0}=1)$) dado que o tratamento foi positivo ($X=1$) e o outcome para o tratamento positivo foi positivo ($Y_{X=1} = 1$).

In [31]:
class CounterfactualSCM:
    def abduct(self, t, y):
        """Computes the value of  U (coffee sensitivity) given the values for treatment (drinking coffee)
        and actual outcome (feel bad in your stomach)"""
        u = (t + y - 1) / (2 * t - 1)
        return  u
    
    def modify(self, t):
        """Modifies the SCM by assigning t to T"""
        return lambda u: t * u + (t - 1) * (u - 1)
    
    def predict(self, u, t):
        """Takes the modified SCM and generates the counterfactual prediction by assigning the actual value of U
        to the modified SCM."""
        return self.modify(t)(u)

In [34]:
coffee = CounterfactualSCM()
t = 1
y = 1
u = coffee.abduct(t=t, y=y)
u

1.0

In [35]:
coffee.predict(u=u, t=0)

0.0